In [31]:
import numpy as np
import scipy as sp
import pandas as pd
from pandas import DataFrame
import matplotlib as mpl
import talib as ta
from matplotlib import pyplot as plt
import matplotlib
from matplotlib.pyplot import plot as plot
from numpy import inf, arange, array, linspace, exp, log, power, pi, cos, sin, radians, degrees
from math import fabs

In [32]:
df = pd.read_csv('data/GBPUSD_60.csv', header=0, index_col='time', parse_dates=True)

In [33]:
# calculate mean
mean = (df['close'] + df['open']) / 2
df['mean'] = mean
df.head(3)

,open,high,low,close,mean
time,,,,,
2018-12-02 22:00:00,1.27211,1.27455,1.27211,1.27352,1.272815
2018-12-02 23:00:00,1.27351,1.27518,1.27351,1.27512,1.274315
2018-12-03 00:00:00,1.27511,1.27665,1.27449,1.27631,1.275710


In [34]:
# calculate distance price between open and close
distance = df['close'] - df['open']
df['distance'] = distance
df.head(3)

,open,high,low,close,mean,distance
time,,,,,,
2018-12-02 22:00:00,1.27211,1.27455,1.27211,1.27352,1.272815,0.00141
2018-12-02 23:00:00,1.27351,1.27518,1.27351,1.27512,1.274315,0.00161
2018-12-03 00:00:00,1.27511,1.27665,1.27449,1.27631,1.275710,0.00120


In [35]:
# calculate volatility point
point = df['distance'] * 10000
# point=point.apply(lambda x: fabs(x))
df['point'] = point
df.head(3)

,open,high,low,close,mean,distance,point
time,,,,,,,
2018-12-02 22:00:00,1.27211,1.27455,1.27211,1.27352,1.272815,0.00141,14.1
2018-12-02 23:00:00,1.27351,1.27518,1.27351,1.27512,1.274315,0.00161,16.1
2018-12-03 00:00:00,1.27511,1.27665,1.27449,1.27631,1.275710,0.00120,12.0


In [36]:
# calculate abs volatility point
point_series = df['point'].apply(lambda x: fabs(x))
df['point_abs'] = point_series
df.head(3)

,open,high,low,close,mean,distance,point,point_abs
time,,,,,,,,
2018-12-02 22:00:00,1.27211,1.27455,1.27211,1.27352,1.272815,0.00141,14.1,14.1
2018-12-02 23:00:00,1.27351,1.27518,1.27351,1.27512,1.274315,0.00161,16.1,16.1
2018-12-03 00:00:00,1.27511,1.27665,1.27449,1.27631,1.275710,0.00120,12.0,12.0


In [37]:
# add column for hour
time = pd.Series(df.index, index=df.index)
hours = time.apply(lambda x: x.hour)
df['hour'] = hours
df.head(2)

,open,high,low,close,mean,distance,point,point_abs,hour
time,,,,,,,,,
2018-12-02 22:00:00,1.27211,1.27455,1.27211,1.27352,1.272815,0.00141,14.1,14.1,22
2018-12-02 23:00:00,1.27351,1.27518,1.27351,1.27512,1.274315,0.00161,16.1,16.1,23


In [38]:
# hourly describe() table
index = []
desc_data = []
for i in range(24):
    desc = df[(df.hour == i)].point_abs.describe()
    desc_data.append((i,desc['mean'], desc['std'], desc['min'], desc['25%'], desc['50%'], desc['75%'], desc['max']))
dd = pd.DataFrame(desc_data, columns=['hour','mean', 'std', 'min', '25%', '50%', '75%', 'max'])
dd = dd.set_index('hour')

dd.to_csv('data/GBPUSD_60_stat.csv', float_format='%.6f')
dd.head(24)

,mean,std,min,25%,50%,75%,max
hour,,,,,,,
0,5.666667,3.960976,0.8,2.900,4.20,7.900,14.6
1,6.390476,5.861050,0.0,2.200,3.80,9.500,20.2
2,5.404762,5.034727,0.2,2.200,4.10,6.500,19.1
3,2.319048,1.810696,0.2,1.000,2.20,3.100,6.9
4,5.738095,3.881685,0.0,2.800,4.80,8.100,15.1
5,5.900000,3.854218,0.0,2.700,5.40,8.700,13.2
6,7.309524,7.170628,0.1,2.700,4.40,6.500,20.6
7,14.523810,15.935429,1.6,4.900,7.30,15.600,62.8
8,16.857143,13.776595,2.2,6.600,14.20,23.300,55.1


# Decision

Best hours for scalping on GBPUSD is UTC 16-22